In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from numpy.random import seed
from collections import deque
#from tensorflow import set_random_seed
import cv2
import random
import numpy as np
seed=1
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
from collections import deque
import time

In [2]:
os.listdir('/kaggle/input/')

['modelupload',
 'pytorch-model',
 'aug-model',
 'modelres50',
 'more-vids',
 'test-video3',
 'test-video1',
 'new-model',
 'alb-model',
 'test-video',
 'test-video2']

In [3]:
#model = tf.keras.models.load_model('../input/modelupload/resnet50.h5')

In [4]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode
from PIL import Image

In [5]:
!pip install omnixai
!pip  install -U kaleido
!pip install plotly
import json
import torch
from torchvision import models, transforms
from PIL import Image as PilImage

from omnixai.data.image import Image
from omnixai.explainers.vision.specific.gradcam.pytorch.gradcam import GradCAM

import plotly.io as pio
pio.renderers.default = "png"
from IPython.display import display # to display images

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.6/756.6 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for hnswlib: filename=hnswlib-0.6.2-cp37-cp37m-linux_x86_64.whl size=2162573 sha256=b0c2a3bfe28842ac5cf958c0759363f02a1dd4006fe672f59b097be3a6f765f7
  Stored in directory: /root/.cache/pip/wheels/67/01/80/9805daef8cd398ceb20003af220f77c4689cab8e43d466481b
Successfully built hnswlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.7 MB/s eta 0:00:00


In [6]:
#img = Image(PilImage.open('dataset/test/normal/6_12600.jpg').convert('RGB'))
# Load the class names
# The preprocessing model
transform = transforms.Compose([
    transforms.Resize((224,224)),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])

In [7]:
model = torch.load('../input/alb-model/pytorch_res18_alb.pth')
#model.eval()

In [8]:
import torch
from torchvision import transforms
import torchvision.models as models
import cv2
import torch.nn.functional as F
import copy


CLASSES = {0:"Collapsed Building", 1:"Fire", 2:"Flood", 3:"Normal"}
BATCH_SIZE = 8
IMG_SIZE = (224, 224)
TRANSFORM_IMG = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMG_SIZE),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])


preprocess = lambda ims: torch.stack([TRANSFORM_IMG(im.to_pil()) for im in ims])
explainer = GradCAM(
    model=model,
    target_layer=model.layer4[-1],
    preprocess_function=preprocess
)



# model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = torch.load('../input/pytorch-model/pytorch_res18.pth')
model.to(device)
#model.load_state_dict(torch.load('checkpoint.pt'))
#model.eval()



for i in os.listdir('../input/test-video3/'):
    print(i)
    videoCapture = cv2.VideoCapture("../input/test-video3/"+i)
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    print(fps)
    size = (int(videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    ps = 25
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    videoWriter = cv2.VideoWriter(str(i)+"_out.mp4", fourcc, fps, size)
    Q = deque(maxlen=int(fps))
    #with torch.no_grad():
    t1 = time.time()
    c=0
    success, frame = videoCapture.read()
    while success:
        c+=1
        frame_copy = copy.deepcopy(frame) 
        frame_copy = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
        image_gc = Image(copy.deepcopy(frame_copy))


        # Explain the top label
        explanations = explainer.explain(image_gc)
        #print(explanations)

        image_tensor = TRANSFORM_IMG(frame_copy)

        image_tensor = image_tensor.unsqueeze(0) 
        test_input = image_tensor.to(device)
        outputs = model(test_input)
        _, predicted = torch.max(outputs, 1)
        probability =  F.softmax(outputs, dim=1)
        top_probability, top_class = probability.topk(1, dim=1)
        predicted = predicted.cpu().detach().numpy()
        predicted = predicted.tolist()[0]
        Q.append(predicted)

        results = np.array(Q).mean(axis=0)
        #i = np.argmax(results)
        #print(Q, results, CLASSES[np.round(results)])
        label =CLASSES[np.round(results)]
        top_probability = top_probability.cpu().detach().numpy()
        top_probability = top_probability.tolist()[0][0]
        percentage = top_probability
        top_probability = '%.2f%%' % (top_probability * 100)
        if percentage < 0.45:
            label="Normal"
#         if label == 'Normal':
#             color = (0, 150, 0)
#         else:
#             color = (0, 0, 150)
#         frame = cv2.putText(frame, label+': '+top_probability, (50, 50), 2, 0.5 ,color, 1 )

        heatmap = explanations.explanations[0]['scores']
        heatmap = cv2.resize(heatmap, (frame.shape[1], frame.shape[0]))
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        superimposed_img  = cv2.addWeighted(heatmap, 0.3, frame, 0.5, 0)
        #plt.imshow(frame+heatmap)
        #print(superimposed_img.shape)
        if label is "Normal":
            color = (0, 150, 0)
            frame = cv2.putText(frame, label+': '+top_probability, (50, 50), 2, 0.5 ,color, 1 )
            videoWriter.write(frame)
        else:
            color = (0, 0, 150)
            superimposed_img = cv2.putText(superimposed_img, label+': '+top_probability, (50, 50), 2, 0.5 ,color, 1 )
            videoWriter.write(superimposed_img)


        
        success, frame = videoCapture.read()
    videoWriter.release()
    
    t2=time.time()
    print('done', t2-t1, c)
                                





gettyimages-1187122876-640_adpp.mp4
25.0


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1033: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.



done 28.37891936302185 682
gettyimages-1421601676-640_adpp.mp4
29.970030000612258


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


done 31.624539136886597 1027
gettyimages-1354415227-640_adpp.mp4
29.97002997002997


...

done 17.83919405937195 598
gettyimages-1309159036-640_adpp.mp4
29.97002997002997


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


done 38.46069121360779 1270
gettyimages-1372117402-640_adpp.mp4
23.976023976023978


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


done 7.611642360687256 244
